In [1]:
# this is where I will evaluate all of the different trained models on the official test set
import keras
import keras.backend as K

# Imports the Google Cloud client library
from google.cloud import storage

Using TensorFlow backend.
/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


ContextualVersionConflict: (requests 2.14.2 (/Users/chloeloughridge/anaconda/lib/python3.5/site-packages), Requirement.parse('requests<3.0.0dev,>=2.18.0'), {'google-api-core'})

In [7]:
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [8]:
# loading the model of interest
model = keras.models.load_model('./audio_cnn_norm_drop_lstm', custom_objects={'FScore2': FScore2})

In [10]:
# load the test set data here from google cloud

# Instantiates a client
storage_client = storage.Client()
bucket = storage_client.get_bucket('mediaeval_data')
VGG_test_data = bucket.blob('mediaeval_data/TestSet/VGG16_test_data.npy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 207, 1583)         6332      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 205, 10)           47500     
_________________________________________________________________
lstm_2 (LSTM)                (None, 205, 207)          180504    
_________________________________________________________________
dropout_1 (Dropout)          (None, 205, 207)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 205, 207)          343620    
_________________________________________________________________
flatten_1 (Flatten)          (None, 42435)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 207)               8784252   
Total para

In [ ]:
# evaluate the model here